In [2]:
# Importamos librerías necesarias
import telebot
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

In [ ]:
# Variables de configuración
TOKEN = ''
CSV_FILE = '..\data\processed\Articulos_LLM5.csv'  # ← tu archivo CSV

# Cargamos el CSV de artículos
articulos_df = pd.read_csv(CSV_FILE)
articulos_df = articulos_df.drop_duplicates()

# Vamos a unir titulo + contenido + fechas + precios + edad
articulos_df['texto_completo'] = (
    articulos_df['titulo'].fillna('') + '. ' +
    articulos_df['contenido'].fillna('') + ' ' +
    articulos_df['fechas_contexto'].fillna('') + ' ' +
    articulos_df['Precios'].fillna('') + ' '
)

In [4]:
import nltk
from nltk.corpus import stopwords

# Asegúrate de tener descargado el paquete de stopwords de NLTK
nltk.download('stopwords')

# Cargamos las stopwords en español
stopwords_es = stopwords.words('spanish')

# Creamos el TF-IDF usando las stopwords en español
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords_es)

# Ahora sí, transformamos
tfidf_matrix = tfidf_vectorizer.fit_transform(articulos_df['texto_completo'])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abdon\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
from datetime import datetime, timedelta
import dateparser
import re

# Inicializamos el bot
bot = telebot.TeleBot(TOKEN)

# --- Nueva función: interpretar fechas en la pregunta ---

def interpretar_fecha_usuario(pregunta):
    pregunta = pregunta.lower()
    hoy = datetime.now()

    if 'hoy' in pregunta:
        return hoy.date(), hoy.date()
    
    elif 'mañana' in pregunta:
        return (hoy + timedelta(days=1)).date(), (hoy + timedelta(days=1)).date()
    
    elif 'semana que viene' in pregunta or 'semana próxima' in pregunta or 'semana siguiente' in pregunta:
        inicio = hoy + timedelta(days=(7 - hoy.weekday()))  # lunes próximo
        fin = inicio + timedelta(days=6)  # domingo de la próxima semana
        return inicio.date(), fin.date()
    
    elif 'esta semana' in pregunta:
        # Desde hoy hasta domingo
        inicio = hoy
        fin = hoy + timedelta(days=(6 - hoy.weekday()))
        return inicio.date(), fin.date()

    elif 'fin de semana' in pregunta or 'sábado' in pregunta or 'domingo' in pregunta:
        # Calcula el próximo sábado y domingo
        dias_hasta_sabado = (5 - hoy.weekday()) % 7
        sabado = hoy + timedelta(days=dias_hasta_sabado)
        domingo = sabado + timedelta(days=1)
        return sabado.date(), domingo.date()
    
    else:
        # No se detectó ninguna fecha relevante
        return None, None

def detectar_ciudad(pregunta):
    ciudades = ["madrid", "barcelona", "malaga", "sevilla", "valencia", "zaragoza"]
    pregunta = pregunta.lower()
    for ciudad in ciudades:
        if ciudad in pregunta:
            return ciudad.capitalize()
    return None

# --- Nueva función: convertir fechas del dataset ---
def preparar_fechas_dataset(df):
    fecha_inicio_parseada = []
    fecha_fin_parseada = []

    for inicio, fin in zip(df['fecha_inicio'].fillna(''), df['fecha_fin'].fillna('')):
        # Parsear fecha inicio
        try:
            if inicio:
                fecha_inicio_dt = dateparser.parse(str(inicio), languages=['es'])
                fecha_inicio_parseada.append(fecha_inicio_dt.date() if fecha_inicio_dt else None)
            else:
                fecha_inicio_parseada.append(None)
        except Exception as e:
            fecha_inicio_parseada.append(None)

        # Parsear fecha fin
        try:
            if fin:
                fecha_fin_dt = dateparser.parse(str(fin), languages=['es'])
                fecha_fin_parseada.append(fecha_fin_dt.date() if fecha_fin_dt else None)
            else:
                fecha_fin_parseada.append(None)
        except Exception as e:
            fecha_fin_parseada.append(None)

    df['fecha_inicio_parseada'] = fecha_inicio_parseada
    df['fecha_fin_parseada'] = fecha_fin_parseada

    return df# Procesamos el dataframe para agregar la columna de fechas
articulos_df = preparar_fechas_dataset(articulos_df)

# --- Función para buscar la mejor respuesta ---
def buscar_respuesta(pregunta_usuario):
    fecha_inicio_consulta, fecha_fin_consulta = interpretar_fecha_usuario(pregunta_usuario)
    ciudad_consulta = detectar_ciudad(pregunta_usuario)

    if fecha_inicio_consulta and fecha_fin_consulta:
        print(f"DEBUG 🛠 Fecha detectada -> Inicio: {fecha_inicio_consulta}, Fin: {fecha_fin_consulta}")
    else:
        print("DEBUG 🛠 No se detectó ninguna fecha en la consulta.")

    if ciudad_consulta:
        print(f"DEBUG 🛠 Ciudad detectada: {ciudad_consulta}")
    else:
        print("DEBUG 🛠 No se detectó ninguna ciudad en la consulta.")

    filtrados = articulos_df.copy()

    # Filtrar por fechas
    if fecha_inicio_consulta and fecha_fin_consulta:
        filtrados = filtrados[
            (filtrados['fecha_inicio_parseada'].notna()) &
            (filtrados['fecha_fin_parseada'].notna()) &
            (filtrados['fecha_inicio_parseada'] <= fecha_fin_consulta) &
            (filtrados['fecha_fin_parseada'] >= fecha_inicio_consulta)
        ]

    # Filtrar por ciudad
    if ciudad_consulta:
        filtrados = filtrados[filtrados['texto_completo'].str.lower().str.contains(ciudad_consulta.lower())]

    if filtrados.empty:
        return "🤔 No encontré actividades para esa ciudad y fecha. ¡Intenta preguntar otra cosa!"

    # TF-IDF búsqueda
    tfidf_matrix_filtrada = tfidf_vectorizer.transform(filtrados['texto_completo'])
    pregunta_vectorizada = tfidf_vectorizer.transform([pregunta_usuario])
    similitudes = cosine_similarity(pregunta_vectorizada, tfidf_matrix_filtrada)

    # Sacamos los índices de las 3 mejores similitudes
    mejores_indices = np.argsort(similitudes[0])[::-1][:3]
    mejores_scores = similitudes[0][mejores_indices]

    print(f"DEBUG 🛠 Mejores scores de similitud: {mejores_scores}")

    # Construir respuesta con máximo 3 actividades
    respuesta = ""
    for idx in mejores_indices:
        articulo = filtrados.iloc[idx]
        respuesta += (
            f"✅ *{articulo['titulo']}*\n"
            f"{articulo['contenido'][:300]}...\n"
            f"💰 Precio: {articulo['Precios']}\n"
            f"🔗 [Más info]({articulo['url']})\n\n"
        )

    return respuesta if respuesta else "🤔 No encontré actividades relacionadas, intenta otra consulta."


# --- Manejadores del bot ---
@bot.message_handler(commands=['start'])
def start(message):
    chat_id = message.chat.id

    # Enviamos la imagen
    with open('..\QHCN.jpg', 'rb') as photo:
        bot.send_photo(chat_id, photo)

    # Enviamos el mensaje de bienvenida
    mensaje_bienvenida = (
        "👋 ¡Hola! Soy tu asistente virtual para ayudarte a buscar actividades.\n\n"
        "Puedes preguntarme cosas como:\n"
        "📌 ¿Qué actividad puedo hacer hoy?\n"
        "📌 A mi hijo/a le gusta el deporte, ¿qué podría hacer esta semana?\n"
        "📌 ¿Qué actividades hay en Sevilla hoy?\n"
        "📌 ¿Qué actividades puedo hacer este fin de semana gratis?\n\n"
        "¡Estoy aquí para ayudarte! 🎯"
    )

    bot.send_message(chat_id, mensaje_bienvenida, parse_mode='Markdown')

@bot.message_handler(func=lambda message: True)
def responder(message):
    consulta = message.text
    respuesta = buscar_respuesta(consulta)
    bot.send_message(message.chat.id, respuesta, parse_mode='Markdown')

# Lanzamos el bot
print("🤖 Bot activo y esperando mensajes...")
bot.polling()

🤖 Bot activo y esperando mensajes...
DEBUG 🛠 Fecha detectada -> Inicio: 2025-05-03, Fin: 2025-05-04
DEBUG 🛠 No se detectó ninguna ciudad en la consulta.
DEBUG 🛠 Mejores scores de similitud: [0.03770497 0.03770497 0.0308019 ]
DEBUG 🛠 Fecha detectada -> Inicio: 2025-04-30, Fin: 2025-04-30
DEBUG 🛠 Ciudad detectada: Madrid
DEBUG 🛠 Mejores scores de similitud: [0.06415191 0.06415191 0.06415191]
DEBUG 🛠 Fecha detectada -> Inicio: 2025-04-30, Fin: 2025-04-30
DEBUG 🛠 Ciudad detectada: Sevilla
DEBUG 🛠 No se detectó ninguna fecha en la consulta.
DEBUG 🛠 Ciudad detectada: Sevilla
DEBUG 🛠 Mejores scores de similitud: [0.28448408 0.24458253 0.24458253]
DEBUG 🛠 No se detectó ninguna fecha en la consulta.
DEBUG 🛠 Ciudad detectada: Madrid
DEBUG 🛠 Mejores scores de similitud: [0.25558435 0.25558435 0.25558435]
DEBUG 🛠 No se detectó ninguna fecha en la consulta.
DEBUG 🛠 No se detectó ninguna ciudad en la consulta.
DEBUG 🛠 Mejores scores de similitud: [0.12918745 0.12918745 0.05095316]
